# Unit 2.3 — Saving Actions in Memory (ActionStep)

This notebook demonstrates the agent execution loop (Reason → Act → Reflect)
using an offline AI agent with persistent memory.

Each user interaction represents an ActionStep containing:
- User input (action trigger)
- Agent response (result)
- Persistent storage in memory and timestamped logs

The project runs fully offline in Google Colab with no API keys or external
inference services required.


In [1]:
# ==========================================================
# 🧩 Colab / Jupyter Environment Repair & Validation (Unit 2.3)
# ==========================================================
import os, sys

print(" Cleaning pip cache …")


print(" Installing stable, compatible packages …")
os.system("""
pip install --no-cache-dir --force-reinstall \
  numpy==1.26.4 \
  scipy==1.12.0 \
  packaging==25.0 \
  fastcore==1.8.0 \
  torch==2.2.2 \
  torchvision==0.17.2 \
  torchaudio==2.2.2 \
  transformers==4.44.0 \
  accelerate==0.29.3 \
  gradio==4.37.2 \
  gradio_client==0.10.1 \
  fastapi==0.115.0 \
  pydantic==2.7.4 \
  sentencepiece==0.2.0 \
  safetensors==0.4.2
""")

print(" Verifying critical imports …\n")
try:
    import numpy, torch, torchvision, transformers, gradio
    from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
    print(f" NumPy        → {numpy.__version__}")
    print(f" SciPy        → {__import__('scipy').__version__}")
    print(f" Torch        → {torch.__version__}")
    print(f" TorchVision  → {torchvision.__version__}")
    print(f" Transformers → {transformers.__version__}")
    print(f" Gradio       → {gradio.__version__}")

    print("\n Environment verified — safe to continue!")
except Exception as e:
    print(" Import validation failed:", e)

print("\n  Please restart runtime now (Runtime ▸ Restart & Run All).")

 Cleaning pip cache …
 Installing stable, compatible packages …
 Verifying critical imports …

 NumPy        → 2.0.2
 SciPy        → 1.16.3
 Torch        → 2.9.0+cpu
 TorchVision  → 0.24.0+cpu
 Transformers → 5.0.0
 Gradio       → 5.50.0

 Environment verified — safe to continue!

  Please restart runtime now (Runtime ▸ Restart & Run All).


In [2]:
# ==========================================================
#  Offline Multi‑Persona Gradio Chat Agent (No API Keys)
# ==========================================================
import os, gradio as gr
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

MODEL_NAME = "distilgpt2"   # lightweight, fully offline

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Fix padding if missing
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})
    model.config.pad_token_id = tokenizer.pad_token_id

agent = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

# -----------------------------
# Persistent memory (ActionSteps)
# -----------------------------
MEM_FILE = "ai_memory.txt"
os.makedirs("chat_logs", exist_ok=True)
memory = open(MEM_FILE).read() if os.path.exists(MEM_FILE) else ""

def save_memory(text):
    with open(MEM_FILE, "w", encoding="utf-8") as f:
        f.write(text)

def save_chat(chat):
    fn = f"chat_logs/chat_{datetime.now():%Y-%m-%d_%H-%M-%S}.txt"
    with open(fn, "w", encoding="utf-8") as f:
        for u, a in chat:
            f.write(f"User: {u}\nAI: {a}\n\n")
    return f" Chat saved → {fn}"

# Persona instructions
PERSONAS = {
    "Friendly": "You are warm, cheerful, and casual.",
    "Teacher": "You explain clearly and provide examples in detail.",
    "Coder": "You focus on concise, commented Python code with short explanations.",
    "Philosopher": "You answer reflectively, exploring ideas deeply."
}

# -------------------------
# Agent Execution Loop
# -------------------------

def chat_ai(user_msg, chat_hist, persona):
    global memory

    # Reason
    prompt = f"""
{PERSONAS[persona]}
Conversation memory:
{memory}

User input:
{user_msg}

AI response:
"""
    # Act
    out = agent(prompt, max_new_tokens=150, temperature=0.7)[0]["generated_text"]
    reply = out.split("AI response:")[-1].strip()

    # Reflect (ActionStep storage)
    step = f"""
[ActionStep | {datetime.now():%Y-%m-%d %H:%M:%S}]

Input: {user_msg}

Output: {reply}
"""
    memory += step
    save_memory(memory)

    chat_hist.append((user_msg, reply))
    return chat_hist, ""

# -------------------------
# Build Gradio UI
# -------------------------
with gr.Blocks() as app:
    gr.Markdown("### Multi-Persona Offline AI Agent (Unit 2.3)")
    persona = gr.Dropdown(label="Persona", choices=list(PERSONAS.keys()), value="Friendly")
    bot = gr.Chatbot(label="Conversation")
    msg = gr.Textbox(label="Message")

    with gr.Row():
        send_btn = gr.Button("Send")
        clr_btn = gr.Button("Clear Memory")
        save_btn = gr.Button("Save Chat")

    send_btn.click(chat_ai, [msg, bot, persona], [bot, msg])
    clr_btn.click(lambda: open(MEM_FILE, "w").close(), None, None)
    save_btn.click(lambda c: save_chat(c), [bot], [])

app.launch()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/tmp/ipython-input-1872069608.py:88: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  bot = gr.Chatbot(label="Conversation")
/tmp/ipython-input-1872069608.py:88: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  bot = gr.Chatbot(label="Conversation")


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cc8627ec1c8a6a28ff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
